In [ ]:
!pip install transformers datasets torch accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers datasets torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import os
from google.colab import drive

# Définir l'appareil
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Chargement du modèle et du tokenizer BARThez
model_name = "moussaKam/barthez"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

In [ ]:

# Chargement du dataset
dataset = load_dataset("LsTam/CQuAE_synthetic")

README.md:   0%|          | 0.00/5.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12304 [00:00<?, ? examples/s]

In [ ]:
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'qid', 'course', 'definition', 'factual', 'synthesis'],
        num_rows: 12304
    })
})


In [ ]:
print(dataset['train'][0])

{'text': 'Jeanne-Marie Phlipon, dite Manon Roland (1754‑1793), est issue de la bourgeoisie. Particulièrement brillante, elle reçoit une éducation de haut niveau. Lorsque survient la Révolution, elle s’implique avec enthousiasme dans le mouvement. Elle y joue un rôle clé, par l’intermédiaire de son mari, Jean‑Marie Roland, élu député puis nommé ministre, en corédigeant notamment tous ses discours. Son influence politique passe en outre par le salon qu’elle organise à Paris, dans la continuité des sociabilités savantes et mondaines du siècle des Lumières. Arrêtée en juin 1793 lors de la chute des Girondins, elle est guillotinée le 8 novembre 1793. Pendant la Révolution, Madame Roland anime un salon où elle reçoit les principales figures des Girondins, à l’image du député Jacques Pierre Brissot.\nBrissot vint nous visiter […]. Il nous fit connaître ceux des députés que d’anciennes relations ou la seule conformité des principes et le zèle de la chose publique réunissaient fréquemment pour 

In [ ]:

def prepare_data(examples):
    inputs = []
    targets = []

    for i in range(len(examples['text'])):
        context = examples['text'][i]
        qid = examples['qid'][i]

        # Créer une entrée distincte pour chaque type de question
        for q_type in ['definition', 'factual', 'synthesis']:
            question_data = examples[q_type][i]
            if question_data and question_data['question']:
                question = question_data['question']

                # Incorporer des marqueurs pour le type de question
                input_text = f"<type:{q_type}> Contexte: {context} QID: {qid}"

                inputs.append(input_text)
                targets.append(question)

    # Tokenisation des entrées
    inputs_tokenized = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    targets_tokenized = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")

    labels = targets_tokenized["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100

    return {
        "input_ids": inputs_tokenized["input_ids"],
        "attention_mask": inputs_tokenized["attention_mask"],
        "labels": labels
    }


In [ ]:
# Appliquer la préparation des données au dataset
prepared_dataset = dataset['train'].map(prepare_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/12304 [00:00<?, ? examples/s]

In [ ]:

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=True,
    report_to=[],
    learning_rate=5e-5
)

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_dataset,
    eval_dataset=prepared_dataset.select(range(1000)),
    tokenizer=tokenizer,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-633df32e936b>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

# Lancement de l'entraînement
trainer.train()

Step,Training Loss,Validation Loss
500,1.699500,1.106910
1000,1.696300,0.962990
1500,1.537100,0.911513
2000,1.481700,0.870804
2500,1.398200,0.835363
3000,1.298800,0.791948
3500,1.463800,0.756478
4000,1.315400,0.733320
4500,1.287800,0.719129
5000,1.082400,0.676366


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Could not locate the best model at ./results/checkpoint-36500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=36912, training_loss=0.8299624347501391, metrics={'train_runtime': 6626.2065, 'train_samples_per_second': 44.565, 'train_steps_per_second': 5.571, 'total_flos': 9.002915534890598e+16, 'train_loss': 0.8299624347501391, 'epoch': 8.0})

In [ ]:
# Sauvegarde du modèle après rechargement
drive.mount('/content/drive')
model.cpu()
model.save_pretrained("/content/drive/MyDrive/generator_question")
tokenizer.save_pretrained("/content/drive/MyDrive/generator_question")

print(" Modèle `generator_question` sauvegardé")

# Vérification de la sauvegarde
model_path = '/content/drive/My Drive/generator_question'

if os.path.exists(model_path):
    print(f"Le modèle est bien sauvegardé {model_path}")

    # Afficher le contenu du répertoire
    files = os.listdir(model_path)
    print("Fichiers trouvés :")
    for file in files:
        print(f"- {file}")
else:
    print("Le modèle n'est pas enregistré")

Mounted at /content/drive
✅ Modèle `BARThez` sauvegardé avec succès sous le nom de `generator_question`!
Le modèle est bien sauvegardé dans : /content/drive/My Drive/generator_question
Fichiers trouvés :
- config.json
- generation_config.json
- model.safetensors
- tokenizer_config.json
- special_tokens_map.json
- sentencepiece.bpe.model
- tokenizer.json


In [ ]:
# Fonction pour générer des questions
def generate_multiple_questions(text, model, tokenizer, device, question_types=['definition', 'factual', 'synthesis']):
    questions = []

    for q_type in question_types:
        input_text = f"<type:{q_type}> Contexte: {text}"

        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

        outputs = model.generate(
            **inputs,
            max_length=128,
            num_return_sequences=1,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            no_repeat_ngram_size=2
        )

        generated_question = tokenizer.decode(outputs[0], skip_special_tokens=True)
        questions.append((q_type, generated_question))

    return questions

# Exemple d'utilisation
text = """
L'intelligence artificielle (IA) est un domaine de l'informatique qui vise à créer des machines capables de simuler l'intelligence humaine. Elle englobe plusieurs sous-domaines, dont l'apprentissage automatique, le traitement du langage naturel et la vision par ordinateur. L'IA a de nombreuses applications pratiques, allant des assistants virtuels aux véhicules autonomes.
"""

# Chargement du modèle et du tokenizer
model_path = '/content/drive/My Drive/generator_question'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Génération des questions
generated_questions = generate_multiple_questions(text, model, tokenizer, device)

# Affichage des questions générées
for q_type, question in generated_questions:
    print(f"{q_type.capitalize()} question: {question}")


Definition question: Qu'est-ce que l'intelligence artificielle?
Factual question: Quels sont les sous-domaines de l'intelligence artificielle mentionnés dans le texte ?
Synthesis question: Pourquoi l'intelligence artificielle (IA) est-elle considérée comme un domaine pertinent pour l’informatique ?


In [ ]:
# Fonction pour générer des questions
def generate_multiple_questions(text, model, tokenizer, device, question_types=['definition', 'factual', 'synthesis']):
    questions = []

    for q_type in question_types:
        input_text = f"<type:{q_type}> Contexte: {text}"

        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

        outputs = model.generate(
            **inputs,
            max_length=128,
            num_return_sequences=1,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            no_repeat_ngram_size=2
        )

        generated_question = tokenizer.decode(outputs[0], skip_special_tokens=True)
        questions.append((q_type, generated_question))

    return questions

# Exemple d'utilisation
text = """
La pollution atmosphérique tue 5,5 millions de personnes par an dans le monde (dont 2,6 millons de décès indirects) selon des chiffres 2016 de la Banque mondiale: elle est est devenue le quatrième facteur de décès prématuré sur Terre. Maladies cardiovasculaires, cancers des poumons, maladies pulmonaires chroniques, infections respiratoires... La pollution de l'air est coupable d'un décès sur dix dans le monde, six fois plus que le paludisme. La cause est l'Homme et son activité, par les industries, le trafic routier, les incinérateurs de déchets, le chauffage individuel et les centrales électriques aux combustibles fossiles. La pollution dans les villes provoque souvent un brouillard de polluants ou smog, souvent révélateurs de la densité de micro-particules et de l'impact du CO2 et autres polluants sur l'environnement.
"""

# Chargement du modèle et du tokenizer
model_path = '/content/drive/My Drive/generator_question'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Génération des questions
generated_questions = generate_multiple_questions(text, model, tokenizer, device)

# Affichage des questions générées
for q_type, question in generated_questions:
    print(f"{q_type.capitalize()} question: {question}")

Definition question: Qu'est-ce que le brouillard de polluants?
Factual question: Combien de personnes sont mortes de la pollution atmosphérique chaque année dans le monde?
Synthesis question: Comment la pollution atmosphérique influence-t-elle la santé publique et l'environnement?
